# **FLOPs For IRv2**

In [2]:
from tensorflow.keras.layers import Input, concatenate, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, Dropout, UpSampling2D, ZeroPadding2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from skimage.transform import resize
import os
import csv
import PIL
import numpy as np
import random
import cv2
# import imutils
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2

In [3]:
HEIGHT = 240
WIDTH = 320
INIT_LR = 0.0001
EPOCHS = 15
TRAIN_PATH = "/kaggle/input/cityscape-csv/cityscape_train.csv"
TEST_PATH = "/kaggle/input/cityscape-csv/cityscape_test.csv"

In [4]:
def downsampling_block(input_tensor, n_filters):
  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(input_tensor)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

def upsampling_block(input_tensor, n_filters, name, concat_with):
  x = UpSampling2D((2, 2), interpolation='bilinear', name=name)(input_tensor)
  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convA")(x)
  x = LeakyReLU(alpha=0.2)(x)

  x = concatenate([x, concat_with], axis=3)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convB")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convC")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

In [5]:

def build(height, width, depth):
  # input
  i = Input(shape=(height, width, depth))

  iresnet = InceptionResNetV2(include_top = False, weights = "imagenet", input_tensor = i)
  # iresnet.summary()


  conv1 = iresnet.get_layer("input_layer").output
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = iresnet.get_layer("activation").output
  conv2 = ZeroPadding2D(((1,0),(1,0)))(conv2)  
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = iresnet.get_layer("activation_3").output
  conv3 = ZeroPadding2D((1,1))(conv3)  
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = iresnet.get_layer("activation_74").output
  conv4 = ZeroPadding2D(((2,1),(2,1)))(conv4)  
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
  

  # bottleneck
#   conv5 = Conv2D(512, (3, 3), padding='same')(pool4)
  conv5 = iresnet.get_layer("activation_161").output
  conv5 = ZeroPadding2D((1,1))(conv5) 
  conv5 = LeakyReLU(alpha=0.2)(conv5)
#   conv5 = Conv2D(512, (3, 3), padding='same')(conv5)
  conv5 = iresnet.get_layer("activation_161").output
  conv5 = ZeroPadding2D((1,1))(conv5)
  conv5 = LeakyReLU(alpha=0.2)(conv5)

  print(conv5.shape,conv4.shape)

  # decoder
  conv6 = upsampling_block(conv5, 256, "up1", concat_with=conv4)
  conv7 = upsampling_block(conv6, 128, "up2", concat_with=conv3)
  conv8 = upsampling_block(conv7, 64, "up3", concat_with=conv2)
  conv9 = upsampling_block(conv8, 32, "up4", concat_with=conv1)

  # output
  o = Conv2D(filters=1, kernel_size=3, strides=(1,1), activation='sigmoid', padding='same', name='conv10')(conv9)

  model = Model(inputs=i, outputs=o)
  return model



In [6]:
model = build(HEIGHT, WIDTH, 3)

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(None, 15, 20, 288) (None, 30, 40, 256)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [7]:
# model.summary()

In [8]:
from tensorflow.keras import layers, models
parameters = model.count_params()

# Manual FLOPs Calculation for Conv2D and Dense Layers
def calculate_flops(model, input_shape=(240, 320, 3)):
    flops = 0

    # Get input size
    h, w, c_in = input_shape
    input_size = h * w * c_in
    
    # Calculate FLOPs for Conv2D layers
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            kernel_h, kernel_w = layer.kernel_size
            c_out = layer.filters
            stride = layer.strides[0]
            
            # Output dimensions after stride
            output_h = (h - kernel_h) // stride + 1
            output_w = (w - kernel_w) // stride + 1
            
            # FLOPs for Conv2D: Output size * kernel size * input channels * output channels
            flops += output_h * output_w * c_out * kernel_h * kernel_w * c_in
            # Update the input dimensions for the next layer
            h, w, c_in = output_h, output_w, c_out
    
    # Calculate FLOPs for Dense layers
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Use layer.input_shape from the model to calculate input units
            input_units = layer.input.shape[1]  # Input dimension to the Dense layer
            output_units = layer.units           # Output dimension from the Dense layer
            
            # FLOPs for Dense: 2 * input_units * output_units (multiply + add operations)
            flops += 2 * input_units * output_units

    return flops

# Calculate FLOPs
flops = calculate_flops(model)

# Print results
print(f"Parameters: {parameters}")
print(f"FLOPs: {flops}")


Parameters: 31159265
FLOPs: 113231474784
